# Tutorial Qiskit Nature [0.7.2] - GSS: Ground State Solvers

Qiskit Nature 0.7.2
Tutorial URL: https://qiskit-community.github.io/qiskit-nature/tutorials/01_electronic_structure.html



## 1 - Struttura Elettronica

### 1.1 - Intro

Solito discorso: hamiltoniana del sistema scomponibile in elettronica e nucleare perché dinamica degli elettroni si esaurisce prima di quella dei nuclei -> Born-Oppenheimer. Le energie elettroniche si trovano risolvendo l'equazione di Schrödinger stazionaria $H_{el} \ket{\psi_{n}} = E_n \ket{\psi_{n}}$.

L'energia di stato fondamentale è data da:
$$
    E_0 = \frac{\bra{\psi_0} H \ket{\psi_0}}{\braket{\psi_0|\psi_0}}
$$

### 1.2 - Dichiarare un **ElectronicStructureProblem**

Il punto di partenza è una soluzione di Hartree-Fock. Qiskit accede alla libreria Python di metodi classici PySCF tramite PySCFDriver. In questo primo esempio si dichiara una molecola di H2 con la distanza di equilibrio di 0.735 angstrom in stato di singoletto e con carica nulla. Qiskit Nature rappresenta il problema elettronico attraverso un oggetto ElectronicStructureProblem, che si produce con il seguente codice:

In [1]:

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

# Per specificare la geometria molecolare si fornice ogni atomo separatamente, divisi da ;
# La lettera indica l'elemento, la posizione è data con coordinate cartesiane
geometry = "H 0 0 0; H 0 0 0.735" 

driver = PySCFDriver(
    atom=geometry,
    basis="sto3g", # 3 gaussiane per approssimare una funzione di Slater
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

problem = driver.run()
print(problem)

#### Più dettagli su **ElectronicStructureProblem** 

L'aspetto più rilevante è l'hamiltoniana interna, in questo caso elettronica: **ElectronicEnergy**. Questa classe è capace di generare l'hamiltoniana di seconda quantizzazione per gli integrali a 1 e 2 corpi, calcolata in precedenza dal codice classico e membro dell'oggetto **problem**. 

In [2]:
hamiltonian = problem.hamiltonian

coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)

Polynomial Tensor
 "+-":
array([[-1.25633907e+00,  3.47700168e-17],
       [ 4.93572504e-17, -4.71896007e-01]])
 "++--":
array([ 6.75710155e-01, -1.00856435e-16,  1.80931200e-01,  6.64581730e-01,
       -3.25210285e-19,  6.98573723e-01])


In [3]:
second_q_op = hamiltonian.second_q_op() # applico il metodo second_q_op() per estrarre l'hamiltoniana di II quantizzazione
print(second_q_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -1.25633907300325 * ( +_0 -_0 )
+ -0.471896007281142 * ( +_1 -_1 )
+ -1.25633907300325 * ( +_2 -_2 )
+ -0.471896007281142 * ( +_3 -_3 )
+ 0.3378550774017582 * ( +_0 +_0 -_0 -_0 )
+ 0.3322908651276483 * ( +_0 +_1 -_1 -_0 )
+ 0.3378550774017582 * ( +_0 +_2 -_2 -_0 )
+ 0.3322908651276483 * ( +_0 +_3 -_3 -_0 )
+ 0.09046559989211571 * ( +_0 +_0 -_1 -_1 )
+ 0.09046559989211571 * ( +_0 +_1 -_0 -_1 )
+ 0.09046559989211571 * ( +_0 +_2 -_3 -_1 )
+ 0.09046559989211571 * ( +_0 +_3 -_2 -_1 )
+ 0.09046559989211571 * ( +_1 +_0 -_1 -_0 )
+ 0.09046559989211571 * ( +_1 +_1 -_0 -_0 )
+ 0.09046559989211571 * ( +_1 +_2 -_3 -_0 )
+ 0.09046559989211571 * ( +_1 +_3 -_2 -_0 )
+ 0.3322908651276483 * ( +_1 +_0 -_0 -_1 )
+ 0.3492868613660083 * ( +_1 +_1 -_1 -_1 )
+ 0.3322908651276483 * ( +_1 +_2 -_2 -_1 )
+ 0.3492868613660083 * ( +_1 +_3 -_3 -_1 )
+ 0.3378550774017582 * ( +_2 +_0 -_0 -_2 )
+ 0.3322908651276483 * ( +_2 +_1 -_1 -_2 )
+ 0.3378550774017582 

L'energia di repulsione nucleare non è inclusa in **second_q_op**, ma è un membro di **hamiltonian** e si richiama facilmente con:

In [4]:
print(hamiltonian.nuclear_repulsion_energy)

0.7199689944489797


#### Ulteriori attributi di **ElectronicStructureProblem**

In [5]:
print(problem.molecule) # le informazioni sulla geometria molecolare fornite all'inizio
print('-----------------------------------------')

print(problem.reference_energy)
print('-----------------------------------------')

print(problem.num_particles)
print('-----------------------------------------')

print(problem.num_spatial_orbitals)

print('-----------------------------------------')
print(problem.basis)

Molecule:
	Multiplicity: 1
	Charge: 0
	Unit: Bohr
	Geometry:
		H	(0.0, 0.0, 0.0)
		H	(0.0, 0.0, 1.3889487015553204)
	Masses:
		H	1
		H	1
-----------------------------------------
-1.1169989967540044
-----------------------------------------
(1, 1)
-----------------------------------------
2
-----------------------------------------
ElectronicBasis.MO


**ElectronicStructureProblem** contiene anche operatori utili per generare osservabili da valutare sui vari stati della molecola. 

### 1.3 - Risolvere il problema elettronico

Per calcolare lo stato fondamentale del problema si utilizza la funzione **GroundStateEigensolver**, maggiori informazioni qui: https://qiskit-community.github.io/qiskit-nature/tutorials/03_ground_state_solvers.html

In [6]:
# Creo un oggetto solver
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [7]:
# Do in pasto al solver il problema 
result = solver.solve(problem)
print(result)

 # da scypy 1.14.0 csr_matrix NON contiene più un membro .H
 # per eseguire il codice occorre sostituire .H con .conjugate().T nella riga 155 di numpy_eigensolver.py


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


## 2 - Problema Nucleare

Sostanzialmente analogo al problema elettronico. Si creano un oggetto **VibrationalStructureProblem** e un Vibrational operator; si danno in pasto al GroundStateEigensolver, colle opzioni di qubit mapping e solver (NumPyMinimumEigensolver). Lo farò più avanti.

## 3 - Ground-State Solvers

Ripartiamo da capo: bisogna definire una molecola con PySCFDriver, quindi scegliere un modo per mappare il problema. 

Per approfondire la questione mapping: https://qiskit-community.github.io/qiskit-nature/tutorials/06_qubit_mappers.html

In [8]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735", # stesso problema di prima
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

es_problem = driver.run()

# di nuovo mappo con Jordan-Wigner

from qiskit_nature.second_q.mappers import JordanWignerMapper

mapper = JordanWignerMapper()

### 3.1 - Solver classico

Bisogna definire il solver che si vuole utilizzare; **NumPyMinimumEigensolver** è un algoritmo classico che diagonalizza l'hamiltoniana. Funziona bene per piccoli sistemi, per cui può essere usato come confronto.

In [9]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

### 3.2 - Solver quantistico (VQE)

Per definire il solver servono tre elementi:
1. Un Estimator primitive (Qiskit Terra) [ep4_primitives] {https://docs.quantum.ibm.com/api/qiskit/primitives}
1. Un ansatz (qui usa UCC, già implementato in Qiskit) [il tipo di eccitazioni è personalizzabile (S, D, SD)]
1. Un ottimizzatore: codice classico che ottimizza i parametri nella forma variazionale {https://qiskit-community.github.io/qiskit-algorithms/apidocs/qiskit_algorithms.optimizers.html}

In [10]:
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator # capire differenza tra Estimator e EstimatorV2
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD 

# chiaramente l'ansatz prende come argomenti le caratteristiche della molecola
ansatz = UCCSD(
    es_problem.num_spatial_orbitals, 
    es_problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        es_problem.num_spatial_orbitals,
        es_problem.num_particles,
        mapper,
    ),
)

vqe_solver = VQE(Estimator(), ansatz, SLSQP()) # Estimator, forma variazionale, ottimizzatore
vqe_solver.initial_point = [0.0] * ansatz.num_parameters


/var/folders/88/gfgbzsfs2m759nlsp4wspv400000gp/T/ipykernel_69712/222947040.py:18: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe_solver = VQE(Estimator(), ansatz, SLSQP())


### 3.3 - Calcoli e risultati

1. Crea l'operatore di seconda quantizzazione
1. Mappa 
1. Esegue l'algoritmo
1. Quando termina (converge) valuta ulteriori osservabili sullo stato determinato

In [13]:
# come prima, creo un oggetto GroundStateEigensolver che contenga il mapper e il solver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver (mapper, vqe_solver)

res = calc.solve(es_problem) 
print(res) # -1.857275030144

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030144
  - computed part:      -1.857275030144
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035695
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388949033206]
    - computed part:      [0.0  0.0  1.388949033206]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000333206]  Total: 0.000000333206
                 (debye): [0.0  0.0  -0.000000846926]  Total: 0.000000846926
 


In [15]:
# comparo con il risultato dell'algoritmo classico

calc = GroundStateEigensolver(mapper, numpy_solver)
res = calc.solve(es_problem) 
print(res) # -1.857275030202 molto molto simili

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388948701555]
    - computed part:      [0.0  0.0  1.388948701555]
  > Dipole moment (a.u.): [0.0  0.0  -0.000000001555]  Total: 0.000000001555
                 (debye): [0.0  0.0  -0.000000003953]  Total: 0.000000003953
 


### 3.4 - Funzioni di Filtering

Of particular importance in the case of vibrational structure calculations.